In [1]:
import torch
from torch import nn
from torch.nn import functional as F
# import dataset
from torch.utils.data import DataLoader, Dataset
# import pad sequence
import numpy as np
import os
import numpy as np
import librosa
from sklearn.metrics import recall_score, f1_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import sys
sys.path.append('D:/NCKH.2025-2026/profHuong/Emotional-Vietnamese-Speech-Based-Depression-Diagnosis-Using-Dynamic-Attention-Mechanism')
#from common import CBAM
avarage = 'micro'
torch.cuda.is_available()

False

In [24]:
class Dual(nn.Module):
    def __init__(self):
        super(Dual, self).__init__()

        # self.feature_extractor1 = nn.Sequential(
        #     # 1st Conv Layer + BatchNorm + ReLU + Pooling
        #     nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
        #     nn.BatchNorm1d(64),
        #     nn.ReLU(),
        #     nn.MaxPool1d(kernel_size=4, stride=4),

        #     # 2nd Conv Layer + BatchNorm + ReLU + Pooling
        #     nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
        #     nn.BatchNorm1d(128),
        #     nn.ReLU(),
        #     nn.MaxPool1d(kernel_size=4, stride=4),

        #     # 3rd Conv Layer + BatchNorm + ReLU + Pooling
        #     nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
        #     nn.BatchNorm1d(128),
        #     nn.ReLU(),
        #     nn.MaxPool1d(kernel_size=4, stride=4),

        #     # 4th Conv Layer + BatchNorm + ReLU + Pooling
        #     nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
        #     nn.BatchNorm1d(128),
        #     nn.ReLU(),
        #     nn.MaxPool1d(kernel_size=4, stride=4)
        # )

        # self.lstm = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        # # out put lstm (batch_size, seq_len, hidden_size) (batch_size, 1, 256)
        # self.fc1 = nn.Linear(256, 5)

        self.feature_extractor2 = nn.Sequential(
            # 1st Conv Layer + BatchNorm + ReLU + Pooling
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=1),  # (128, 251) -> (128, 251, 64)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),  # (128, 251, 64) -> (64, 125, 64)

            # 2nd Conv Layer + BatchNorm + ReLU + Pooling
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=1),  # (64, 125, 64) -> (64, 125, 64)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(4, 4), stride=(4, 4)),  # (64, 125, 64) -> (16, 31, 64)

            # 3rd Conv Layer + BatchNorm + ReLU + Pooling
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=1),  # (16, 31, 64) -> (16, 31, 128)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(4, 4), stride=(4, 4)),  # (16, 31, 128) -> (4, 7, 128)

            # 4th Conv Layer + BatchNorm + ReLU + Pooling
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=1),  # (4, 7, 128) -> (4, 7, 128)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(4, 4), stride=(4, 4))  # (4, 7, 128) -> (1, 1, 128)
        )

        # Fully Connected Layer
        self.fc2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),  # 1*1*128 -> 256
            nn.ReLU(),
            nn.Linear(512, 7),  # 256 -> 5
        )

        # self.truefc = nn.Softmax(dim=1)
        self.gru = nn.GRU(input_size=128, hidden_size=256, bidirectional=False, batch_first=True)
        self.fc3 = nn.Linear(7, 7)


    def forward(self, mfcc):
        # wave_form = self.feature_extractor1(wave_form)  # Pass through the sequential feature extractor

        # # LSTM expects input of shape (batch_size, seq_len, input_size)
        # wave_form = wave_form.permute(0, 2, 1)  # Reshape to (batch_size, seq_len, input_size)
        # wave_form, _ = self.lstm(wave_form)

        # wave_form = wave_form[:, -1, :]

        # wave_form = self.fc1(wave_form)

        mfcc = self.feature_extractor2(mfcc)
        #mfcc = self.cbam(mfcc)
        mfcc = mfcc.squeeze().unsqueeze(1)

        mfcc,_ = self.gru(mfcc)
        mfcc = self.fc2(mfcc)
        # x = torch.cat((wave_form, mfcc), dim=1)

        mfcc = self.fc3(mfcc)

        return mfcc

In [ ]:
n_mfcc = 128
window_size = 2048
strides = 512
window_size_stft = 1024
window = np.hanning(window_size_stft)

emotion_map = {
    'W': 0,  # Anger
    'L': 1,  # Boredom
    'A': 2,  # Anxiety
    'F': 3,  # Happiness
    'T': 4,  # Sadness
    'E': 5,  # Disgust
    'N': 6   # Neutral
}

def load_emodata(link, sr = 16000, duration = 5):
    # Load the audio file
    wave_form, _ = librosa.load(path=link, sr=sr)
    # label = father folder name
    filename = os.path.basename(link)
    emo_code = filename[5] 
    labels = emotion_map.get(emo_code, -1) 

    if len(wave_form) < sr * duration:
        # zero pad the audio if its less than 5 seconds
        wave_form = np.pad(wave_form, (0, sr * duration - len(wave_form)), 'symmetric')
        mfcc1 = librosa.feature.mfcc(y=wave_form, sr=8000, n_mfcc=n_mfcc, n_fft=window_size, hop_length=strides)
        # stft1 = librosa.core.spectrum.stft(wave_form, n_fft=window_size_stft, hop_length=256, window=window)
        # spect1 = 2 * np.abs(stft1) / np.sum(window)
        return np.array([mfcc1]), np.array([labels])

    elif len(wave_form) == sr * duration:
        mfcc2 = librosa.feature.mfcc(y=wave_form, sr=8000, n_mfcc=n_mfcc, n_fft=window_size, hop_length=strides)
        # stft2 = librosa.core.spectrum.stft(wave_form, n_fft=window_size_stft, hop_length=256, window=window)
        # spect2 = 2 * np.abs(stft2) / np.sum(window)
        # return the audio as it is
        return np.array([mfcc2]), np.array([labels])


    else:
        wave_segments = []
        _labels = [labels] * (len(wave_form) // (sr * duration) + 1)
        for i in range(0, len(wave_form), sr * duration):
            wave_segments.append(wave_form[i:i + sr * duration])

        # If the last segment is less than 5 seconds, then pad it with the last 5 seconds of the second last segment
        len_wave_segments_last = len(wave_segments[-1])
        padding = sr * duration - len_wave_segments_last
        temp = np.append(wave_segments[-2][sr * duration - padding:], wave_segments[-1])
        wave_segments[-1] = temp

        mfcc_seg = librosa.feature.mfcc(y=np.array(wave_segments), sr=8000, n_mfcc=n_mfcc, n_fft=window_size, hop_length=strides)
        # stft_seg = librosa.core.spectrum.stft(wave_segments, n_fft=window_size_stft, hop_length=256, window=window)
        # spect_seg = 2 * np.abs(stft_seg) / np.sum(window)

        return np.array(mfcc_seg) , np.array(_labels)

In [26]:
data_folder = "c:/Users/GIGABYTE/Downloads/archive/wav"

In [27]:
def load_data(data_folder):
    # waves = []
    mfccs = []
    labels = []
    for root, dirs, files in os.walk(data_folder):
        for file in files:
            if file.endswith(".wav"):
                link = os.path.join(root, file)
                mfcc, label = load_emodata(link)

                # waves.extend(wave)
                mfccs.extend(mfcc)
                labels.extend(label)

    return np.array(mfccs), np.array(labels)

In [28]:
mfccs, labels = load_data(data_folder)

In [29]:
len(mfccs), len(labels)

(560, 560)

In [30]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [31]:
class EmoDataset(Dataset):
    def __init__(self, mfccs, labels):

        self.mfccs = mfccs
        self.labels = labels

    def __len__(self):
        return len(self.mfccs)

    def __getitem__(self, index):
        return self.mfccs[index], self.labels[index]

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
EMO_CLASSES = {label: i for i, label in enumerate(np.unique(labels))}

In [34]:
EMO_CLASSES

{np.int64(0): 0,
 np.int64(1): 1,
 np.int64(2): 2,
 np.int64(3): 3,
 np.int64(4): 4,
 np.int64(5): 5,
 np.int64(6): 6}

In [35]:
for idx, (train_idx, test_idx) in enumerate(skf.split(mfccs, labels)):

    # wave_train, wave_test = waves[train_idx], waves[test_idx]
    mfcc_train, mfcc_test = mfccs[train_idx], mfccs[test_idx]
    train_labels, test_labels = labels[train_idx], labels[test_idx]

    train_labels = [EMO_CLASSES[label] for label in train_labels]
    test_labels = [EMO_CLASSES[label] for label in test_labels]


    train_dataset = EmoDataset(mfcc_train, train_labels)
    test_dataset = EmoDataset(mfcc_test, test_labels)

    print("Train data: ", Counter(train_labels))
    print("Test data: ", Counter(test_labels))


Train data:  Counter({0: 102, 1: 65, 6: 64, 4: 63, 3: 56, 2: 55, 5: 43})
Test data:  Counter({0: 26, 1: 16, 4: 16, 3: 15, 6: 15, 2: 14, 5: 10})
Train data:  Counter({0: 102, 1: 64, 6: 63, 4: 63, 3: 57, 2: 56, 5: 43})
Test data:  Counter({0: 26, 1: 17, 6: 16, 4: 16, 3: 14, 2: 13, 5: 10})
Train data:  Counter({0: 102, 1: 65, 4: 64, 6: 63, 3: 57, 2: 55, 5: 42})
Test data:  Counter({0: 26, 6: 16, 1: 16, 4: 15, 3: 14, 2: 14, 5: 11})
Train data:  Counter({0: 103, 1: 65, 6: 63, 4: 63, 3: 57, 2: 55, 5: 42})
Test data:  Counter({0: 25, 6: 16, 1: 16, 4: 16, 2: 14, 3: 14, 5: 11})
Train data:  Counter({0: 103, 1: 65, 6: 63, 4: 63, 3: 57, 2: 55, 5: 42})
Test data:  Counter({0: 25, 4: 16, 6: 16, 1: 16, 2: 14, 3: 14, 5: 11})


In [36]:
epochs = 100
batch_size = 32
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
EMO_CLASSES = {label: i for i, label in enumerate(set(labels))}

In [38]:
acc_kfold = []
f1_kfold = []
recall_kfold = []
precision_kfold = []
data_kfold = []
labels_preds_kfold = []

In [39]:
for idx, (train_idx, test_idx) in enumerate(skf.split(mfccs, labels)):

    mfcc_train, mfcc_test = mfccs[train_idx], mfccs[test_idx]
    train_labels, test_labels = labels[train_idx], labels[test_idx]

    train_labels = [EMO_CLASSES[label] for label in train_labels]
    test_labels = [EMO_CLASSES[label] for label in test_labels]


    train_dataset = EmoDataset(mfcc_train, train_labels)
    test_dataset = EmoDataset(mfcc_test, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = Dual()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_acc = 0
    best_f1 = 0
    best_recall = 0
    best_precision = 0

    acc = []
    f1 = []
    recall = []
    precision = []
    best_labels_preds = []

    for epoch in range(epochs):
        tt_loss = 0
        model.train()
        for mfcc, label in train_loader:

            mfcc = mfcc.unsqueeze(1).to(device)
            label = label.to(device)

            optimizer.zero_grad()
            output = model(mfcc)

            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            tt_loss += loss.item()

        print("K-Fold: ", idx, "Epoch: ", epoch, "Train loss: ", tt_loss / len(train_loader))

        model.eval()

        with torch.no_grad():
            all_labels = []
            all_preds = []

            for mfcc, label in test_loader:

                mfcc = mfcc.unsqueeze(1).to(device)
                label = label.to(device)

                output = model(mfcc)
                _, predicted = torch.max(output, 1)

                all_labels.extend(label.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())


            validation_acc = accuracy_score(all_labels, all_preds)
            recall_val = recall_score(all_labels, all_preds, average="micro")
            f1_val = f1_score(all_labels, all_preds, average="micro")
            precision_val = precision_score(all_labels, all_preds, average="micro")

            acc.append(validation_acc)
            f1.append(f1_val)
            recall.append(recall_val)
            precision.append(precision_val)


            if validation_acc > best_acc:
                best_acc = validation_acc
                best_recall = recall_val
                best_f1 = f1_val
                best_precision = precision_val
                best_labels_preds = [all_labels, all_preds]

                # torch.save(model.state_dict(), f"best_model_{idx}.pth")

            print(f"K-Fold: {idx}, Epoch: {epoch}, Accuracy: {validation_acc}, Recall: {recall_val}, F1: {f1_val}, Precision: {precision_val}")

    data_kfold.append([acc, f1, recall, precision])

    acc_kfold.append(best_acc)
    f1_kfold.append(best_f1)
    recall_kfold.append(best_recall)
    precision_kfold.append(best_precision)
    labels_preds_kfold.append(best_labels_preds)


K-Fold:  0 Epoch:  0 Train loss:  1.7537360021046229
K-Fold: 0, Epoch: 0, Accuracy: 0.33035714285714285, Recall: 0.33035714285714285, F1: 0.33035714285714285, Precision: 0.33035714285714285
K-Fold:  0 Epoch:  1 Train loss:  1.4095699020794459
K-Fold: 0, Epoch: 1, Accuracy: 0.30357142857142855, Recall: 0.30357142857142855, F1: 0.30357142857142855, Precision: 0.30357142857142855
K-Fold:  0 Epoch:  2 Train loss:  1.355669115270887
K-Fold: 0, Epoch: 2, Accuracy: 0.39285714285714285, Recall: 0.39285714285714285, F1: 0.39285714285714285, Precision: 0.39285714285714285
K-Fold:  0 Epoch:  3 Train loss:  1.1674457149846214
K-Fold: 0, Epoch: 3, Accuracy: 0.375, Recall: 0.375, F1: 0.375, Precision: 0.375
K-Fold:  0 Epoch:  4 Train loss:  1.01793800507273
K-Fold: 0, Epoch: 4, Accuracy: 0.48214285714285715, Recall: 0.48214285714285715, F1: 0.48214285714285715, Precision: 0.48214285714285715
K-Fold:  0 Epoch:  5 Train loss:  0.8872150012425014
K-Fold: 0, Epoch: 5, Accuracy: 0.5446428571428571, Recal

KeyboardInterrupt: 

In [40]:
len(data_kfold)

0

In [41]:
len(labels_preds_kfold)

0

In [42]:
def print_classification_report(labels_preds_kfold):
    for idx, data in enumerate(labels_preds_kfold):
        print(f"K-Fold: {idx}")
        print(classification_report(data[0], data[1], target_names=EMO_CLASSES.keys()))

In [43]:
def plot_metrics(data_kfold):
    num_folds = len(data_kfold)
    plt.figure(figsize=(20, 5))

    for idx, data in enumerate(data_kfold):
        acc, f1, recall, precision = data
        plt.subplot(1, num_folds, idx + 1)
        plt.plot(acc, label='Accuracy')
        plt.plot(recall, label='Recall')
        plt.plot(f1, label='F1')
        plt.plot(precision, label='Precision')
        plt.title(f"K-Fold {idx + 1}")
        plt.xlabel('Epochs')
        plt.ylabel('Metrics')
        plt.ylim(0.0, 1.0)  # Set y-axis range from 0.0 to 1.0
        plt.legend()
        plt.grid(True)

    plt.tight_layout()  # Adjust subplots to fit into figure area
    plt.show()

In [44]:
acc_kfold

[]

In [ ]:
np.mean(acc_kfold),np.mean(f1_kfold),np.mean(precision_kfold),np.mean(f1_kfold)

In [ ]:
print_classification_report(labels_preds_kfold)

In [ ]:
# average classification report
def average_classification_report(labels_preds_kfold):
    all_labels = []
    all_preds = []
    for data in labels_preds_kfold:
        all_labels.extend(data[0])
        all_preds.extend(data[1])
    print(classification_report(all_labels, all_preds, target_names=EMO_CLASSES.keys()))

In [ ]:
average_classification_report(labels_preds_kfold)

In [ ]:
# plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns


def plot_confusion_matrix(labels_preds_kfold):
    all_labels = []
    all_preds = []
    for data in labels_preds_kfold:
        all_labels.extend(data[0])
        all_preds.extend(data[1])

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=EMO_CLASSES.keys(), yticklabels=EMO_CLASSES.keys())
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
plot_confusion_matrix(labels_preds_kfold)

In [ ]:
def compute_ua_wa(labels_preds_kfold):
    true_labels = np.array([])
    predicted_labels = np.array([])

    # Gộp tất cả nhãn thực và nhãn dự đoán từ các fold
    for data in labels_preds_kfold:
        true_labels = np.append(true_labels, data[0])
        predicted_labels = np.append(predicted_labels, data[1])

    # Tính UA (Unweighted Accuracy)
    ua = accuracy_score(true_labels, predicted_labels)

    # Tính WA (Weighted Accuracy)
    wa = []
    for label in np.unique(true_labels):
        i_true = true_labels[true_labels == label]
        i_predicted = predicted_labels[true_labels == label]

        wa.append(np.sum(i_true == i_predicted) / len(i_true))
    wa = np.mean(wa)
    return ua, wa

In [ ]:
ua, wa = compute_ua_wa(labels_preds_kfold)
print(f"Unweighted Accuracy: {ua}, Weighted Accuracy: {wa}")